<a href="https://colab.research.google.com/github/roberthsu2003/Transformer/blob/main/%E5%AF%A6%E6%88%B0%E9%81%8B%E7%94%A8/%E6%83%85%E6%84%9F%E5%88%86%E6%9E%90/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.


## Text Classification
- 使用模型:    
google-bert/bert-base-chinese

- 使用資料集:  
roberthsu2003/data_for_classification

### 載入套件

In [2]:
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer,DataCollatorWithPadding,AutoModelForSequenceClassification,Trainer,TrainingArguments

### 載入資料集
- roberthsu2003/data_for_classification

In [3]:
#載入資料集
datasets = load_dataset("roberthsu2003/data_for_classification",cache_dir='./my_dataset')
datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/452 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/211k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6988 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

In [4]:
#分詞
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

def tokenize_function(item):
    '''
    只要先分詞,不要現在轉成tensor,也不用padding,轉成tensor和自動padding,由DataCollator來作
    由於是classfication,所以目標得到的欄位是'input_ids','token_type_ids','attention_mask'另外還要的是'label'
    '''
    tokenized = tokenizer(item['review'],
                          max_length=318,
                          truncation=True)
    tokenized['label'] = item['label']
    return tokenized

tokenized_datasets = datasets.map(function=tokenize_function,batched=True,
                                  remove_columns=datasets['train'].column_names)
tokenized_datasets

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 777
    })
})

In [5]:
#可以發現input_ids長度是不一樣的
len(tokenized_datasets['train']['input_ids'][0]),len(tokenized_datasets['train']['input_ids'][1])

(135, 119)

### 建立DataCollator

In [6]:
#使用DataCollatorWithPadding,讓長度一樣,並要求傳出pytorch tensor
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,return_tensors="pt")
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

### 載入預訓練模型

In [7]:

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-chinese")
#trainer會自動將模型放到gpu上,所以不用手動cuda()


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 建立評估函數

In [8]:
#建立評估函數
import evaluate
acc_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')
#acc_metric = evaluate.load('evaluate-main/metrics/accuracy/accuracy.py')
#f1_metric = evaluate.load('evaluate-main/metrics/f1/f1.py')

In [9]:
#定義評估函數
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions,references=labels)
    f1 = f1_metric.compute(predictions=predictions,references=labels)
    acc.update(f1)
    return acc

### TrainingArguments

In [10]:
#建立TrainingArguments,多加一些參數

train_args = TrainingArguments(
  output_dir='./checkpoints',
  report_to="none",  # Disable W&B logging,在colab訓練時,要關閉
  per_device_train_batch_size=32, #訓練時每個gpu的batch size
  per_device_eval_batch_size=32, #評估時每個gpu的batch size
  num_train_epochs=3, #訓練的epochs,預設是3
  logging_steps=100, #每100個batch log一次,預設是500
  eval_strategy="epoch", #評估策略,每一趟評估1次,預設是no
  save_strategy="epoch", #儲存策略,每一趟儲存1次,預設是no
  learning_rate=2e-5, #學習率,預設是5e-5
  weight_decay=0.01, #權重衰減,預設是0.01
  metric_for_best_model="f1", #最佳模型的指標,預設是eval_loss
  greater_is_better=True, #指標是否越大越好,預設是False
  load_best_model_at_end=True, #訓練完後,是否載入最佳模型,預設是False
  )
#目前的TrainingArguments有預設值,可以直接使用


In [ ]:
### 建立Trainer

In [11]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    compute_metrics=eval_metric
)


### 訓練模型


In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.232000,0.219688,0.897040,0.923664
2,0.161600,0.239881,0.912484,0.935115
3,0.115000,0.262856,0.920206,0.942379


TrainOutput(global_step=657, training_loss=0.17729699774964214, metrics={'train_runtime': 1134.9885, 'train_samples_per_second': 18.471, 'train_steps_per_second': 0.579, 'total_flos': 3382609172141280.0, 'train_loss': 0.17729699774964214, 'epoch': 3.0})

In [13]:
#模型評估
trainer.evaluate()

{'eval_loss': 0.2628559470176697,
 'eval_accuracy': 0.9202059202059202,
 'eval_f1': 0.9423791821561338,
 'eval_runtime': 12.9233,
 'eval_samples_per_second': 60.124,
 'eval_steps_per_second': 1.934,
 'epoch': 3.0}

### 上傳模型,評估資料和tokenizer

In [14]:
from huggingface_hub import login
login()

In [15]:
model_name = "roberthsu2003/models_for_classfication"
trainer.push_to_hub(model_name) #由於有設./checkpoints,所以自動產生checkpoints的repo,也會自動上傳評估至repo

#同時要上傳tokenizer
tokenizer.push_to_hub(model_name)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/roberthsu2003/models_for_classfication/commit/a7237164d89fde1ea33a5111b4ba74ea9a6b0925', commit_message='Upload tokenizer', commit_description='', oid='a7237164d89fde1ea33a5111b4ba74ea9a6b0925', pr_url=None, repo_url=RepoUrl('https://huggingface.co/roberthsu2003/models_for_classfication', endpoint='https://huggingface.co', repo_type='model', repo_id='roberthsu2003/models_for_classfication'), pr_revision=None, pr_num=None)